In [1]:
notas = input("Insira uma lista de elementos separados por espaço: ")
list_notas  = notas.split()
print(list_notas)

Insira uma lista de elementos separados por espaço: 00440111682 00440111689
['00440111682', '00440111689']
